In [88]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import glob
from scipy.stats import t

In [46]:
# acquire all csv path
csv_files = glob.glob('D:/mentor/return_file/*.csv')

stock_returns = pd.DataFrame()

# iterate reading and concating
for file in csv_files:
    stock_return = pd.read_csv(file)
    stock_returns = pd.concat([stock_returns, stock_return])

stock_returns

,Stkcd,Trddt,Dsmvosd,Dretwd,Markettype,Trdsta
0,1,2002-01-04,17067373.40,-0.011429,4,1
1,1,2002-01-07,16968718.06,-0.005780,4,1
2,1,2002-01-08,16940530.82,-0.001661,4,1
3,1,2002-01-09,16757313.76,-0.010815,4,1
4,1,2002-01-10,16644564.81,-0.006728,4,1
...,...,...,...,...,...,...
999995,300033,2010-01-12,976953.60,0.010987,16,1
999996,300033,2010-01-13,1034208.00,0.058605,16,1
999997,300033,2010-01-14,1047648.00,0.012995,16,1
999998,300033,2010-01-15,1102080.00,0.051956,16,1


In [3]:
# Load the CSV files into pandas DataFrames
risk_free_rate = pd.read_csv('D:/mentor/risk free/TRD_Nrrate.csv')
fama_factors = pd.read_csv('D:/mentor/three factor/STK_MKT_THRFACDAY.csv')

# Convert daily risk-free rate to decimal
risk_free_rate['Nrrdaydt'] = risk_free_rate['Nrrdaydt'] / 100
risk_free_rate['Nrrmtdt'] = risk_free_rate['Nrrmtdt'] / 100

risk_free_rate.rename(columns={'Clsdt': 'date'}, inplace=True)

In [18]:
risk_free_rate['date'] = pd.to_datetime(risk_free_rate['date'])
risk_free_rate['month'] = risk_free_rate['date'].dt.to_period('M')

In [8]:
def data_preprocess(stock_returns: pd.DataFrame, fama_factors: pd.DataFrame):
    # Remove rows with marketType equal to 2 or 8
    stock_returns = stock_returns[~stock_returns['Markettype'].isin([2, 8])]

    # Filter fama_factors for marketType equal to P9709
    fama_factors = fama_factors[fama_factors['MarkettypeID'] == 'P9709']


    if 'Trddt' in stock_returns.columns:
        # Remove rows with Trdsta equal to 2 or 16
        stock_returns = stock_returns[~stock_returns['Trdsta'].isin([2, 16])]
        stock_returns.rename(columns={'Trddt': 'date'}, inplace=True)
        fama_factors.rename(columns={'TradingDate': 'date'}, inplace=True)
    else:
        stock_returns.rename(columns={'Trdmnt': 'month'}, inplace=True)
        fama_factors.rename(columns={'TradingMonth': 'month'}, inplace=True)

    stock_returns.rename(columns={'Stkcd': 'stock'}, inplace=True)


    return stock_returns,fama_factors


In [5]:
def merge_df(merge_on, stock_returns: pd.DataFrame, fama_factors: pd.DataFrame,risk_free_rate: pd.DataFrame):
    # Merge the DataFrames on a common date column
    merged_data = pd.merge(stock_returns, risk_free_rate, on = merge_on)
    merged_data = pd.merge(merged_data, fama_factors, on = merge_on)
    return merged_data

In [101]:
from functools import partial

def reg(data: pd.DataFrame, col_y: list, col_x: list):
    if len(col_y) == 2:
        y = data[col_y[0]] - data[col_y[1]]
    else:
        y = data[col_y]
    X = sm.add_constant(data[col_x])
    lm = sm.OLS(y, X)
    result = lm.fit()

    return result

def reg_res(data: pd.DataFrame, col_y: list, col_x: list):
    # Perform regression and calculate residuals

    res = reg(data,col_y,col_x).resid

    # Return standard deviation of residuals
    return res.std()

def reg_const(data: pd.DataFrame, col_y: list, col_x: list):

 return reg(data,col_y,col_x).params['const']


In [12]:
# read monthly return file
return_M = pd.read_csv('D:/mentor/return_file/month/TRD_Mnth.csv')
fama_factors_M = pd.read_csv('D:/mentor/three factor/STK_MKT_THRFACMONTH.csv')


return_M, fama_factors_M = data_preprocess(return_M,fama_factors_M)


E:\python\lib\site-packages\pandas\core\frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
fama_factors_M

,MarkettypeID,month,RiskPremium1,SMB1,HML1
3,P9709,2002-01,-0.105090,-0.032669,0.019441
15,P9709,2002-02,0.026290,0.010387,-0.026765
27,P9709,2002-03,0.060025,0.027245,-0.007949
39,P9709,2002-04,0.041014,0.022340,0.018600
51,P9709,2002-05,-0.086833,-0.001085,-0.014776
...,...,...,...,...,...
1562,P9709,2012-08,-0.025539,0.076941,-0.010138
1575,P9709,2012-09,0.018826,-0.022135,-0.020717
1588,P9709,2012-10,-0.009209,0.016114,0.007724
1601,P9709,2012-11,-0.053451,-0.062710,0.049620


In [15]:
return_M

,stock,month,Msmvosd,Mretwd,Markettype
0,1,2002-01,14840581.49,-0.140408,4
1,1,2002-02,14305023.94,-0.036087,4
2,1,2002-03,15545262.47,0.086700,4
3,1,2002-04,16207662.60,0.042611,4
4,1,2002-05,15728479.53,-0.029565,4
...,...,...,...,...,...
213532,603766,2012-11,537600.00,-0.091892,1
213533,603766,2012-12,696000.00,0.294643,1
213534,603993,2012-10,1596000.00,NaN,1
213535,603993,2012-11,1312000.00,-0.177945,1


In [30]:
risk_free_rate

,Nrr1,date,Nrrdata,Nrrdaydt,Nrrwkdt,Nrrmtdt,month
0,NRI01,2002-01-01,2.25,0.000061,0.0428,0.001856,2002-01
1,NRI01,2002-01-02,2.25,0.000061,0.0428,0.001856,2002-01
2,NRI01,2002-01-03,2.25,0.000061,0.0428,0.001856,2002-01
3,NRI01,2002-01-04,2.25,0.000061,0.0428,0.001856,2002-01
4,NRI01,2002-01-05,2.25,0.000061,0.0428,0.001856,2002-01
...,...,...,...,...,...,...,...
4013,NRI01,2012-12-27,3.00,0.000081,0.0569,0.002466,2012-12
4014,NRI01,2012-12-28,3.00,0.000081,0.0569,0.002466,2012-12
4015,NRI01,2012-12-29,3.00,0.000081,0.0569,0.002466,2012-12
4016,NRI01,2012-12-30,3.00,0.000081,0.0569,0.002466,2012-12


In [25]:
risk_free_rate.groupby('month').mean()

,Nrrdata,Nrrdaydt,Nrrwkdt,Nrrmtdt
month,,,,
2002-01,2.250000,0.000061,0.042800,0.001856
2002-02,2.172857,0.000059,0.041343,0.001793
2002-03,1.980000,0.000054,0.037700,0.001635
2002-04,1.980000,0.000054,0.037700,0.001635
2002-05,1.980000,0.000054,0.037700,0.001635
...,...,...,...,...
2012-08,3.000000,0.000081,0.056900,0.002466
2012-09,3.000000,0.000081,0.056900,0.002466
2012-10,3.000000,0.000081,0.056900,0.002466


In [64]:
# data_ = merge_df('month',return_M, fama_factors_M,risk_free_rate)
data_ = pd.merge(return_M, fama_factors_M, on = 'month')


In [65]:
data_['month'] = pd.to_datetime(data_['month']).dt.to_period('M')

In [66]:
data_ = pd.merge(data_, risk_free_rate[['month','Nrrmtdt']], on = 'month')

In [67]:
data_ .set_index(['stock','month'],inplace = True)

In [71]:
data_ = data_.drop_duplicates()

In [72]:
data_

,,Msmvosd,Mretwd,Markettype,MarkettypeID,RiskPremium1,SMB1,HML1,Nrrmtdt
stock,month,,,,,,,,
1,2002-01,1.484058e+07,-0.140408,4,P9709,-0.105090,-0.032669,0.019441,0.001856
2,2002-01,4.936053e+06,-0.072659,4,P9709,-0.105090,-0.032669,0.019441,0.001856
3,2002-01,3.627303e+05,-0.475000,4,P9709,-0.105090,-0.032669,0.019441,0.001856
4,2002-01,8.697980e+05,-0.052202,4,P9709,-0.105090,-0.032669,0.019441,0.001856
5,2002-01,1.551891e+06,-0.197292,4,P9709,-0.105090,-0.032669,0.019441,0.001856
...,...,...,...,...,...,...,...,...,...
601988,2010-08,5.921370e+08,-0.069832,1,P9709,0.013317,0.087314,-0.075616,0.001856
601989,2010-08,1.785525e+07,0.207827,1,P9709,0.013317,0.087314,-0.075616,0.001856
601991,2010-08,5.840408e+07,-0.037657,1,P9709,0.013317,0.087314,-0.075616,0.001856


In [48]:
# stock_returns,fama_factors  = data_preprocess(stock_returns,fama_factors)
stock_returns['date'] = pd.to_datetime(stock_returns['date'])
fama_factors['date'] = pd.to_datetime(fama_factors['date'])
merged_data = merge_df('date',stock_returns,fama_factors,risk_free_rate)

In [49]:
# Group data by 'order_book_id' and 'month', and apply regression function to multiple columns
col_y = ['Dretwd','Nrrdaydt']
col_x = ['RiskPremium1', 'SMB1', 'HML1']

IVOL = merged_data.groupby(['stock','month']).apply(partial(reg_res, col_y=col_y, col_x=col_x))
IVOL

stock   month  
1       2002-01    0.009048
        2002-02    0.011071
        2002-03    0.013884
        2002-04    0.015339
        2002-05    0.011621
                     ...   
601988  2006-09    0.006962
        2006-10    0.007544
        2006-11    0.015700
        2006-12    0.025434
601991  2006-12    0.211663
Length: 119349, dtype: float64

In [73]:
data_['IVOL'] = IVOL

data_

,,Msmvosd,Mretwd,Markettype,MarkettypeID,RiskPremium1,SMB1,HML1,Nrrmtdt,IVOL
stock,month,,,,,,,,,
1,2002-01,1.484058e+07,-0.140408,4,P9709,-0.105090,-0.032669,0.019441,0.001856,0.009048
2,2002-01,4.936053e+06,-0.072659,4,P9709,-0.105090,-0.032669,0.019441,0.001856,0.007335
3,2002-01,3.627303e+05,-0.475000,4,P9709,-0.105090,-0.032669,0.019441,0.001856,NaN
4,2002-01,8.697980e+05,-0.052202,4,P9709,-0.105090,-0.032669,0.019441,0.001856,0.020072
5,2002-01,1.551891e+06,-0.197292,4,P9709,-0.105090,-0.032669,0.019441,0.001856,0.024494
...,...,...,...,...,...,...,...,...,...,...
601988,2010-08,5.921370e+08,-0.069832,1,P9709,0.013317,0.087314,-0.075616,0.001856,NaN
601989,2010-08,1.785525e+07,0.207827,1,P9709,0.013317,0.087314,-0.075616,0.001856,NaN
601991,2010-08,5.840408e+07,-0.037657,1,P9709,0.013317,0.087314,-0.075616,0.001856,NaN


In [75]:
data_['forward_1m_returns'] = data_.groupby('stock')['Mretwd'].shift(-1)
data_ = data_.swaplevel('stock', 'month').sort_index(level = 0)
data_

Msmvosd    Mretwd  Markettype MarkettypeID  RiskPremium1  \
month   stock                                                                  
2002-01 1       14840581.49 -0.140408           4        P9709     -0.105090   
        2        4936053.04 -0.072659           4        P9709     -0.105090   
        3         362730.31 -0.475000           4        P9709     -0.105090   
        4         869797.99 -0.052202           4        P9709     -0.105090   
        5        1551890.91 -0.197292           4        P9709     -0.105090   
...                     ...       ...         ...          ...           ...   
2012-12 603333    707227.20  0.233560           1        P9709      0.145256   
        603366   1540000.00  0.252033           1        P9709      0.145256   
        603399    786297.60  0.296761           1        P9709      0.145256   
        603766    696000.00  0.294643           1        P9709      0.145256   
        603993   1666000.00  0.283669           1        P9709      0.145256   

                    SMB1      HML1   Nrrmtdt      IVOL  forward_1m_returns  
month   stock                                                               
2002-01 1      -0.032669  0.019441  0.001856  0.009048           -0.036087  
        2      -0.032669  0.019441  0.001856  0.007335            0.021002  
        3      -0.032669  0.019441  0.001856       NaN           -0.079365  
        4      -0.032669  0.019441  0.001856  0.020072            0.067050  
        5      -0.032669  0.019441  0.001856  0.024494            0.113253  
...                  ...       ...       ...       ...                 ...  
2012-12 603333  0.011473  0.020950  0.002466       NaN                 NaN  
        603366  0.011473  0.020950  0.002466       NaN                 NaN  
        603399  0.011473  0.020950  0.002466       NaN                 NaN  
        603766  0.011473  0.020950  0.002466       NaN                 NaN  
        603993  0.011473  0.020950  0.002466       NaN                 NaN  

[239681 rows x 10 columns]

In [78]:
data_ = data_.dropna()

In [76]:
def group(x, k):
    return(pd.qcut(x, k, labels = np.arange(1,k+1)))

In [79]:
data_['group'] = data_.groupby('month')['IVOL'].apply(lambda x: group(x, k = 5))

data_[100000:100005]

E:\python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Msmvosd    Mretwd  Markettype MarkettypeID  RiskPremium1  \
month   stock                                                                
2009-05 2119    798536.35 -0.054017           4        P9709      0.051924   
        2120    504559.93  0.001454           4        P9709      0.051924   
        2121   1867436.51  0.334558           4        P9709      0.051924   
        2122   3992960.00 -0.020354           4        P9709      0.051924   
        2123   3957072.78  0.126786           4        P9709      0.051924   

                   SMB1      HML1   Nrrmtdt      IVOL  forward_1m_returns  \
month   stock                                                               
2009-05 2119   0.030207  0.003924  0.001856  0.017698           -0.067350   
        2120   0.030207  0.003924  0.001856  0.034994           -0.010477   
        2121   0.030207  0.003924  0.001856  0.026088           -0.100406   
        2122   0.030207  0.003924  0.001856  0.016818           -0.036785   
        2123   0.030207  0.003924  0.001856  0.030870           -0.076704   

              group  
month   stock        
2009-05 2119      2  
        2120      5  
        2121      4  
        2122      2  
        2123      5

In [80]:
group_returns_ew = data_.groupby(['month', 'group'])['forward_1m_returns'].mean().unstack()
group_returns_ew

group,1,2,3,4,5
month,,,,,
2002-01,0.020845,0.028753,0.030644,0.035832,0.051319
2002-02,0.042459,0.048064,0.051668,0.058936,0.075581
2002-03,0.054081,0.051720,0.050160,0.052190,0.038771
2002-04,-0.090756,-0.083652,-0.091848,-0.085036,-0.091390
2002-05,0.132366,0.132580,0.114635,0.114867,0.107421
...,...,...,...,...,...
2011-08,-0.094923,-0.113426,-0.116945,-0.133765,-0.152754
2011-09,0.031729,0.041003,0.037111,0.042796,0.048534
2011-10,-0.028498,-0.036102,-0.029633,-0.042173,-0.058797


In [83]:
column_means = group_returns_ew.mean()
column_means

group
1    0.020314
2    0.016872
3    0.016693
4    0.015220
5    0.012807
dtype: float64

In [81]:
def value_weighted_avg_return(group):
    weights = group['Msmvosd'] / group['Msmvosd'].sum()
    return (group['forward_1m_returns'] * weights).sum()

group_returns_vw = data_.groupby(['month', 'group']).apply(value_weighted_avg_return).unstack()
group_returns_vw

group,1,2,3,4,5
month,,,,,
2002-01,0.012290,0.024485,0.027422,0.032267,0.043942
2002-02,0.032346,0.042242,0.044338,0.050267,0.066866
2002-03,0.042824,0.039480,0.040833,0.041436,0.029484
2002-04,-0.085112,-0.082954,-0.092759,-0.078942,-0.086614
2002-05,0.143086,0.150479,0.125581,0.136998,0.113519
...,...,...,...,...,...
2011-08,-0.092118,-0.117360,-0.116470,-0.129250,-0.155277
2011-09,0.028126,0.034676,0.025926,0.024318,0.047220
2011-10,-0.032434,-0.060489,-0.045677,-0.058490,-0.060579


In [85]:
column_means_vw = group_returns_vw.mean()
column_means_vw

group
1    0.014925
2    0.015818
3    0.014225
4    0.014961
5    0.012852
dtype: float64

In [95]:
def group_difference_ttest(group):
    group_5 = group[5]
    group_1 = group[1]
    diff = group_5.mean() - group_1.mean()

    # Calculate the standard errors
    var_5 = group_5.var()
    var_1 = group_1.var()
    cov = group_5.cov(group_1)
    n_5 = len(group_5)
    n_1 = len(group_1)
    se = np.sqrt(var_5  + var_1 - 2 * cov)

    # Calculate the t-statistic
    t_statistic = diff / se

    p_value = 2 * (1 - t.cdf(np.abs(t_statistic), df = n_5 - 2))
    return pd.Series({'Diff': diff, 'T-Statistic': t_statistic, 'P-Value': p_value})


In [96]:
 # = group_returns_ew.groupby('month').apply(group_difference_ttest)
# test_data = group_returns_ew.apply(group_difference_ttest, axis=1)
test_data = group_difference_ttest(group_returns_ew)
test_data

Diff          -0.007507
T-Statistic   -0.744931
P-Value        0.457055
dtype: float64

In [102]:
def reg_cons(data: pd.DataFrame, y, col_x: list):

    X = sm.add_constant(data[col_x])
    lm = sm.OLS(y, X)
    result = lm.fit()

    return result

In [103]:
# col_y_M = data_['Mretwd','Nrrmtdt']
rf = data_['Nrrmtdt']
# 对每一列进行回归并减去
group_returns_const = group_returns_ew.sub(rf,axis = 0).apply(lambda x:reg_cons(data_,x,col_x = col_x), axis=0).params['const']
group_returns_const

group
1    0.012617
2    0.011299
3    0.011554
4    0.010783
5    0.011221
dtype: float64

In [104]:
group_returns_const_vw = group_returns_vw.sub(rf,axis = 0).apply(lambda x:reg_cons(data_,x,col_x = col_x), axis=0).params['const']
group_returns_const_vw

group
1    0.008598
2    0.011029
3    0.011705
4    0.012108
5    0.012619
dtype: float64

In [ ]:
# Subtract group_returns_vw[5] from group_returns_vw[1]
group_returns_diff = group_returns_vw[5] - group_returns_vw[1]

# Perform regression on the subtracted returns
t_stat = reg_cons(data_, group_returns_diff, col_x=col_x).tvalues['const']